In [17]:
import pandas as pd
import numpy as np
import plotly.express as px

In [18]:
def load_dataset():
    
    df = pd.read_csv(r"googleplaystore.csv")
    return df

a = load_dataset()
a.head(10)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
5,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
6,Smoke Effect Photo Maker - Smoke Editor,ART_AND_DESIGN,3.8,178,19M,"50,000+",Free,0,Everyone,Art & Design,"April 26, 2018",1.1,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,36815,29M,"1,000,000+",Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
9,Kids Paint Free - Drawing Fun,ART_AND_DESIGN,4.7,121,3.1M,"10,000+",Free,0,Everyone,Art & Design;Creativity,"July 3, 2018",2.8,4.0.3 and up


In [19]:
def print_summarize(dataset):
    
    print("Info")
    print(dataset.info(), '\n')
    print(dataset.describe(), '\n')
    print(dataset.shape)

print_summarize(a)

Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB
None 

            Rating
count  9367.000000
mean      4.193338
std       0.537431
min       1.000000
25%       4.000000
50%       4.300000
75%       4.500000

In [20]:
def clean_dataset(df):
    df["App"] = df["App"].apply(lambda x: str(x).replace('"', "").replace("&", "-"))
    df.drop_duplicates(subset=['App'], inplace=True)
    df.drop(10472, axis=0, inplace=True)
    df["Reviews"] = df["Reviews"].apply(lambda x: str(x).replace("M", "000000")).astype(float)
    df["Type"] = df["Type"].apply(lambda x: str(x).replace("nan", "Free").replace("0", "Free"))
    # Cleaning text
    df['Size'] = df['Size'].apply(lambda x: str(x).replace("M", ""))
    df['Size'] = df['Size'].apply(lambda x: str(x).replace("+", ""))
    df['Size'] = df['Size'].apply(lambda x: str(x).replace(",", ""))
    df['Size'] = df['Size'].apply(lambda x: "0" if x == "Varies with device" else x)
    
    for i, size in df['Size'].items():
        if size.endswith("k"):
            df.loc[i, 'Size'] = float(size[:-1]) / 1024
        elif size.endswith("M"):
            df.loc[i, 'Size'] = float(size[:-1])
        else:
            df.loc[i, 'Size'] = float(size)

    df["Installs"] = df["Installs"].apply(lambda x: str(x).replace("+", "").replace(",", "").replace("Free", "0").replace(".", "")).astype(int)
    df["Price"] = df["Price"].astype(str).apply(lambda x: x.replace("$", "0").replace("Everyone", "0")).astype(float)
    df["Genres"] = df["Genres"].apply(lambda x: str(x).replace("&", "-").replace(";", " "))

    return df
clean_data = clean_dataset(a)


In [21]:
def print_histograms(dataset):
    dataset['Rating'] = pd.to_numeric(dataset['Rating'], errors='coerce')
    fig1 = px.histogram(dataset.sort_values(by="Rating", ascending=False), title="Rating of Category", x='Category', y='Rating', barmode='group')
    fig2 = px.histogram(dataset.sort_values(by="Reviews", ascending=False), title="Reviews of Category", x='Category', y='Reviews', barmode='group')
    fig3 = px.histogram(dataset.sort_values(by="Size", ascending=False), title="Size of Category", x='Category', y='Size', barmode='group')

    
    
    fig1.show()
    fig2.show()
    fig3.show()

print_histograms(clean_data)


In [22]:
def compute_correlations_matrix(dataset):
    fig = px.imshow(dataset.select_dtypes(np.number).corr(), text_auto=True)
    fig.show()
    
compute_correlations_matrix(clean_data)

In [23]:
def print_scatter_matrix():
    rating = clean_data[["Rating", "Reviews"]]
    reviews_size = clean_data[["Reviews", "Size"]]
    reviews_installs = clean_data[["Reviews", "Installs"]]
    price_size = clean_data[["Price", "Size"]]

    fig_1 = px.scatter_matrix(rating)
    fig_2 = px.scatter_matrix(reviews_size)
    fig_3 = px.scatter_matrix(reviews_installs)
    fig_4 = px.scatter_matrix(price_size)

    fig_1.show()
    fig_2.show()
    fig_3.show()
    fig_4.show()
print_scatter_matrix()

In [24]:
def most_popular_family(df):
    family = df[(df['Category'] == 'FAMILY') & (df['Price'] > 0)]
    
    sorting = family.sort_values(by='Installs', ascending=False).head(20)
    
    fig = px.bar(sorting, x='Installs', y='App', orientation='h', title='Top 20 paid app family')
    fig.show()
most_popular_family(clean_data)

In [25]:
def genres_paid_pie(df):
    family = df[(df['Category'] == 'FAMILY') & (df['Price'] > 0)]
    
    sorting = family.sort_values(by='Installs', ascending=False).head(20)
    genres = sorting.sort_values(by='Genres').head(20)
    
    fig = px.pie(genres, values='Installs', names='Genres', title='Top 20 paid app family')
    fig.show()
genres_paid_pie(clean_data)

In [26]:
def installs_of_category(df):
    fig = px.histogram(df.sort_values(by="Installs", ascending=False), title="Installs of Category", x='Category', y='Installs', barmode='group')
    fig.show()
installs_of_category(clean_data)

In [27]:
def installs_of_category_pie(df):
    fig = px.pie(df, values='Installs', names='Category', title='Installs of Category')
    fig.show()

installs_of_category_pie(clean_data)

In [28]:
def mean_of_category(df):
    a = df.groupby('Category')['Price'].mean().reset_index()
    
    fig = px.bar(a, x='Category', y='Price', title='Mean of Category', color='Category')
    fig.show()
mean_of_category(clean_data)

In [29]:
def expencive_app(df):
    expencive = df.groupby(by="Category")["Price"].max().sort_values(ascending=False)
    fig = px.bar(expencive, x=expencive.index, y=expencive.values, title="Most expensive per category app", color=expencive.index)
    fig.show()
expencive_app(clean_data)

